<a href="https://colab.research.google.com/github/geekevgin/-Python/blob/main/BEL_Ev_nlp_course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Загрузка и установка библиотек и dataset**

In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from sklearn.model_selection import train_test_split

In [7]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)
import warnings
warnings.filterwarnings("ignore")

In [9]:
df = load_dataset("blinoff/medical_qa_ru_data")

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
df

DatasetDict({
    train: Dataset({
        features: ['date', 'categ', 'theme', 'desc', 'ans', 'spec10'],
        num_rows: 190335
    })
})

In [11]:
df['train'].shape

(190335, 6)

In [12]:
df['train'][5]

{'date': '28 Сентября 2016, 22:21',
 'categ': 'Другое',
 'theme': 'Д-Димер',
 'desc': 'Добрый вечер, на на 22-й недели беременности после проведения операции на ноге Д-Димер был 1,9. На 24-й неделе снизился до 1, 4. Является ли это нормой при беременности или необходимо принимать клексан?',
 'ans': 'Выложите фото анализа,нам нужно знать референсные значения вашей лаборатории.',
 'spec10': ''}

**препроцессинг данных** **и подгоотовка к обучению**

In [13]:
MIN_QUESTION_LEN = 20
MAX_QUESTION_LEN = 1000
MIN_ANSWER_LEN = 50
MAX_ANSWER_LEN = 1000
DELIMITER = "[DEL]"
QUESTION = "[QUE]"
ANSWER = "[ANS]"

In [14]:
def clear_text(text):
    clr_text = text.lower().strip()

    # replacing everything with space except (0-9, а-Я, ".", "?", "!", ",", ":", ";")
    clr_text = re.sub(r"[^0-9а-яА-Я?.!,':;\s]+", " ", clr_text)
    clr_text = re.sub(r";\n", " " + DELIMITER + " ", clr_text)
    clr_text = re.sub(r"\s", " ", clr_text)

    return clr_text.strip()

In [15]:
questions = []
answers = []

question_len = []
answer_len = []
for row in tqdm_notebook(df["train"]):
    q = clear_text(row["desc"])
    a = clear_text(row["ans"])
    if MIN_QUESTION_LEN <= len(q) <= MAX_QUESTION_LEN:
        ans = a.split(DELIMITER)
        a = max(ans, key=len)

        if MIN_ANSWER_LEN <= len(a) <= MAX_ANSWER_LEN:
            questions.append(q)
            answers.append(a)

            question_len.append(len(q))
            answer_len.append(len(a))

  0%|          | 0/190335 [00:00<?, ?it/s]

In [16]:
len(questions), len(answers)

(139627, 139627)

In [15]:
questions[:5]

['ларипронт 20 талеток,через каждые 2 3 часа.очень понравились,но пока принимала,чувствовала себя хорошо. закончились все сиптомы опять проявились. нигде не найду как долго их можно принимать. скажите пожалуйста.',
 'здравствуйте, я на 7 8 неделе беременности. с 6 недели нет аппетита, просто вообще ничего не лезет из еды, а ведь какие то две недели назад я ела с хорошим аппетитом ...                          так вот я просто могу ничего не есть, так как просто вооооообще ничего не лезет!! иногда приходится есть просто так но в малых порциях так как желудок да т уже о себе знать и он кажется опустош н . так вот в обед сегодня я съела йогурт без всякого аппетита и потом порцию наверно как для реб нка и то не доела, и начало чуть мутить. и так всегда . аппетита нет уже с 6 недели.. что со мной??',
 'завтра иду с утра сдавать кровь ттг, т4св, кальцитонин, можно после этого через 30 минут пойти в зал и заниматься спортом?',
 'мне прописали пить аллохол. врач написала пить за 30 минут до еды

In [16]:
answers[:5]

['что вы им лечите? длительность приема ларипронта индивидуальна и устанавливается лечащим врачом. ',
 ' здравствуйте. это ранний токсикоз. если у вас сильная тошнота, частая рвота и вы вообще ничего кушать не можете  именно не можете, а не не хотите, т к кушать все  таки надо, вы должны это понимать. пусть понемногу, пусть детскую дозу, но почаще , обратитесь к гинекологу, он предложит стац. лечение. ',
 ' здравствуйте , да, попейте сладкого чая перед залом хотя бы после сдачи анализов и съешьте фрукт ',
 'препарат принимается после еды. уточните это  у своего лечащего врача. ',
 'выложите фото анализа,нам нужно знать референсные значения вашей лаборатории.']

In [17]:
data = []
for q, a in tqdm_notebook(zip(questions, answers), total=len(questions)):
    line = f"{QUESTION} {q} {ANSWER} {a}"
    data.append(line)

  0%|          | 0/139627 [00:00<?, ?it/s]

In [18]:
train, test = train_test_split(data, test_size=0.15)

In [19]:
test[:5]

['[QUE] с утра появился не большой зуд во влагалище, вечером жение [ANS] здравствуйте                        нужно сдать общий мазок на флору                        пцр на скрытые инфекции                        пока можно ванночки с ромашкой от зуда применять ',
 '[QUE] ломка голоса у девушек и симптомы ломка голоса? во сколько лет  ломка  заканчивается [ANS] изменение тембра голоса. мутация голоса у девочек менее заметна: гортань увеличивается лишь на 1 3, голос понижается на 1 2 тона, постепенно теряя детские свойства и становясь женственным. обычно у детей, проживающих в умеренном климате, изменения голоса начинаются в 13 16 лети продолжаются от месяца до 2 3 лет. ',
 '[QUE] здравствуйте, если консультация возможна, помогите, пожалуйста. полчаса назад сходил у туалет по большому, ощутил резкую боль, а затем заметил обильное кровотечение из ануса, после похода ощущается боль в той области. выпяченные узлы замечал давно  порядка 3 5 лет  во время при ма душа или ванны, но не предавал

In [19]:
train_path = "train_medical_qa_ru_data.txt"
test_path = "test_medical_qa_ru_data.txt"

In [20]:
def build_text_files(data, path):
    with open(path, "w", encoding="utf-8") as f:
        res = " ".join([s.strip() for s in data])
        print("Dataset length:", len(res))
        f.write(res)

In [21]:
build_text_files(train, train_path)

Dataset length: 58202150


In [22]:
build_text_files(test,test_path)

Dataset length: 10301092


In [23]:
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [25]:
special_tokens_dict = {"additional_special_tokens": [QUESTION, ANSWER]}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [26]:
def create_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128
    )

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

In [27]:
train_dataset, test_dataset, data_collator = create_dataset(train_path, test_path, tokenizer)

In [28]:
training_args = TrainingArguments(
    "phrase",
    evaluation_strategy="epoch",
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy="no",
    report_to="none",
)

In [31]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.6 MB/s eta 0:00:00


In [33]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.30.1
Uninstalling transformers-4.30.1:
  Successfully uninstalled transformers-4.30.1
Found existing installation: accelerate 0.20.3
Uninstalling accelerate-0.20.3:
  Successfully uninstalled accelerate-0.20.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.30.1-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)


In [34]:
exit()

In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.319600,3.216282
2,3.243100,3.180428


TrainOutput(global_step=17546, training_loss=3.337911543437941, metrics={'train_runtime': 8224.9982, 'train_samples_per_second': 25.597, 'train_steps_per_second': 2.133, 'total_flos': 1.3752975458304e+16, 'train_loss': 3.337911543437941, 'epoch': 2.0})

In [33]:
output_model = "111_medical_qa_ru_data"

In [34]:
trainer.save_model(output_model)

In [35]:
model = AutoModelForCausalLM.from_pretrained(output_model, local_files_only=True)
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

In [36]:
replace_words_list = [
    "здравствуйте", "спасибо", "добрый день", "добрый вечер"
]

stopwords_list = [
    "пользователь поблагодарил", "поблагодарил за ответ", "за ответ на сумму", "текст благодарности"
]

DEFAULT_ANSWER = "Не могу ответить на Ваш вопрос"

In [37]:
def generate_text(prefix, threshold_len=20):
    tokens = tokenizer(prefix, return_tensors="pt").to("cuda")
    size = tokens["input_ids"].shape[1]

    output = model.generate(
        **tokens, 
        do_sample=False,
        max_length=size+100, 
        early_stopping=True,
        length_penalty=2.0,
        repetition_penalty=8., 
        temperature=0.8,
        num_beams=3,
        no_repeat_ngram_size=5
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]

    if ANSWER not in result:
        answer = DEFAULT_ANSWER
    else:
        answer = result.split(ANSWER)[1]
        answer = answer.split(QUESTION)[0]

        for s in stopwords_list:
            if s in answer.lower():
                answer = DEFAULT_ANSWER
                break

        for s in replace_words_list:
            answer = re.sub(s, "_", answer, re.IGNORECASE) 

        found = False
        substrings = answer.split("_")
        for s in substrings:
            if len(s) >= threshold_len:
                answer = s
                found = True
                break
        if not found:
            answer = max(substrings, key=len)


    answer = " ".join(answer.split())
    answer = answer.strip(" .,!:;")
    for letter in "?.!,:;":
        answer = re.sub(r"\s*[\\" + letter + r"]\s*", f"{letter} ", answer)

    answer = ". ".join(map(lambda x: x.capitalize(), (s for s in answer.split(". "))))
    return answer

In [39]:
print(generate_text("Что принять при головной боли?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Вам нужно обратиться к невропатологу для осмотра и назначения адекватного лечения. Если не сможете найти специалиста в вашем городе рассмотрите вариант такой консультации в скайпе


In [40]:
print(generate_text("Чем сбить температуру ребенку?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Не могу ответить на ваш вопрос


In [41]:
print(generate_text("Выкидыш на 12й недели беременности?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Да, это может быть беременность. Вам нужно обратиться к гинекологу и выполнить узи органов малого таза для исключения эндометриоза шейки матки или полипа эндометрия в матке с контрастированием хлоргексидином 1 2 раза в день во время овуляции до 7 дней после окончания менструации, исключить воспалительный процесс цервикального канала при мастурбации. Удачи вам


In [42]:
print(generate_text("Чем лечить насморк?"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Вам нужно обратиться к лор врачу для осмотра и назначения адекватного лечения. Если у вас нет жалоб на носоглотку, то можете закапывать в нос сосудосуживающие капли или орошать горло мирамистином с антибиотиком цефтриаксоном 1 раз в день 3 4 дня по 10 капель 2 раза в день до еды 5 6 дней после еды 7 8 дней перерыв между приемами пищи 12 14 дней, а затем повторить курс через месяц при
